In [1]:
import gensim
from gensim.models import Word2Vec, FastText
# import tensorflow
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

In [2]:
TRAIN = 5000
SEG_SIZE = 100
TRAIN_SEG = round(TRAIN / SEG_SIZE)
LABELED_USERS = 10
EMBEDDING_DIM = 128
DATA_PATH = 'FraudedRawData'

In [3]:
def create_embeddings(sentences, vector_size=128, window=5):
    model = FastText(sentences=sentences, vector_size=vector_size, window=window)
    model.save("fasttext.model")
    return model

In [4]:
def load_user_data(dir_path=DATA_PATH):
    df = pd.DataFrame()
    for i in range(40):
        tmp = pd.read_csv(dir_path + '/User' + str(i), sep=' ', header=None)
        df = pd.concat([df, tmp], axis=1)
    df = df.T.reset_index(drop=True)
    seg_df = pd.DataFrame()
    for i in range(150):
        seg_df = pd.concat([seg_df, df.iloc[:, i*100:i*100 + 100].apply(lambda x: ",".join(x.astype(str)).split(','), axis=1)],
                           axis=1, names=[str(i)])
    seg_df = seg_df.T.reset_index(drop=True).T
    return df, seg_df

In [5]:
def get_top_k_probas(probas, k=5):
    return np.partition(probas, -k, axis=1)[:, -k:]

In [6]:
def get_top_k_classes(probas, k=5):
    return np.argpartition(probas, -k, axis=1)[:, -k:]

In [8]:
def get_metrics(y_true, y_pred):
    try:
        y_true = y_true.values.flatten().tolist()
    except:
        pass
    print(f'Acc: {accuracy_score(y_true, y_pred)}')
    print(f'Precision: {precision_score(y_true, y_pred)}')
    print(f'Recall: {recall_score(y_true, y_pred)}')
    print(f'F1 Score: {f1_score(y_true, y_pred)}')
    conf_mat = confusion_matrix(y_true, y_pred)
    tp, tn = conf_mat[1, 1], conf_mat[0, 0]
    classification_score = tn + 9*tp
    print(f'Classification Score: {classification_score}')
    final_grade = min(100, 95 * 3*classification_score/4575) # + 0.3 * report_grade
    # print(f'Final Grade: {final_grade}')
    return classification_score
    # return preds